In [1]:
import numpy as np
import gurobipy as gb

# TSDAP
## Time Dependent Activity Scheduling Problem

$\tau_i(t_i)$ is the time required to complete activity $i$ at time $t_i$.  
$[e_i, l_i]$ are the earliest start time and latest end time of activity $i$.  
$\rho_i(t_i)$ is the resource consumption of activity $i$ at time $t_i$.  
$Q$ is the total resource capacity  
$\theta_i(t_i) = t_i + \tau_i(t_i)$ is the completion time function.  

$$\min_{t_j} \theta_n(t_n) = \min_{t_j} t_n + \tau_n(t_n)$$

subject to

$$\theta_i(t_i) \le t_{i+1}$$
$$\sum_{i=1}^{n} \rho_i(t_i) \le Q$$
$$t_i \in [e_i, l_i]$$

We assume the FIFO property: $\theta_i(t) \le \theta_i(t')$ if $t \le t'$

In [75]:
resources = 100
n_activities = 20
eps = 0.1
t_start = 0
t_end = 10
n_steps = int((t_end - t_start) / eps)
t = np.linspace(t_start, t_end, n_steps, endpoint=False) + eps / 2

In [68]:
class Discretizer:
    def __init__(self, min, max, steps):
        self.min = min
        self.max = max
        self.steps = steps
        self.step_size = (max - min) / steps
        self.half_size = self.step_size / 2

    def index_to_value(self, index):
        return self.min + index * self.step_size + self.half_size

    def value_to_index(self, value):
        return int((value - self.min) / self.step_size) 

discretizer = Discretizer(t_start, t_end, n_steps)

In [83]:
def my_tau(start):
    return np.sin(start) + 1


def my_rho(start):
    return np.cos(start) + 1

In [ ]:
# https://stackoverflow.com/questions/65613295/tuple-like-lexographical-max-in-numpy
def lexargmax_by_partition(x):
    view = np.ndarray(x.shape[0], dtype=[('', x.dtype)] * x.shape[1], buffer=x)
    return np.argpartition(view, -1)[-1]


In [84]:
from dataclasses import dataclass
from collections.abc import Callable

@dataclass
class Activity:
    start: float
    end: float
    duration: Callable[[float], float]
    resources: Callable[[float], float]
    name: str = None

    def completion_time(self, start):
        return start + self.duration(start)

activities = [Activity(i, 2*(i+1), my_tau, my_rho) for i in range(n_activities)]

In [85]:
tdasp = gb.Model()
x = tdasp.addMVar((n_activities, n_steps), vtype=gb.GRB.BINARY)

In [3]:
assignment = gb.Model()
assignment.modelSense = gb.GRB.MINIMIZE #declare mimization

# assignment.setParam('OutputFlag', 0) suppress outputs, equivalent of xpress setControl('outputlog', 0)

X = assignment.addVars( [(i,j) for i in range(4) for j in range(4)], vtype=gb.GRB.BINARY) #this way of declare vars does not allow to work with matrix multiplication (X is a tuple dict)

I = range(4)
J = range(4)

costs = np.array([[13,	4,	7, 6],
                	[1,	11,	5, 4],
                  [6,	7,	2, 8],
                  [1,	3,	5, 9]])


for i in I:
  assignment.addConstr( gb.quicksum(X[i,j] for j in J) <= 1) #quicksum is the equivalent to xp.Sum

for j in J:
  assignment.addConstr( gb.quicksum(X[i,j] for i in I) == 1 )

assignment.setObjective( 
    gb.quicksum( costs[i,j]*X[i,j]   for j in J for i in I)
    )   

assignment.optimize() #equivalent to solve() for xpress


Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x6f06b897
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 1.100000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      11.0000000   11.00000  0.00%     -    0s

Explored 1 nodes (